In [25]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random
import statsmodels.api as sm
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel("data.xlsx")
data.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0
2,Tony Ferguson,Donald Cerrone,Dan Miragliotta,2019,"Chicago, Illinois, USA",Red,False,Lightweight,3,0,...,3,6,1,14,Orthodox,180.34,193.04,155.0,36.0,35.0
3,Jimmie Rivera,Petr Yan,Kevin MacDonald,2019,"Chicago, Illinois, USA",Blue,False,Bantamweight,3,0,...,1,0,0,6,Orthodox,162.56,172.72,135.0,26.0,29.0
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",Blue,False,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0


### Apenas a categoria Peso-Pesados ('Heavyweights')

In [3]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [4]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

### Blue = 0
### Red = 1

In [5]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [6]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)
data_heavy_util.dropna(inplace=True)

In [7]:
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.00,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.00,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.00,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.00,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0
75,Alistair Overeem,Aleksei Oleinik,1,0,Heavyweight,5,0,2,0,4.25,...,6,0,0,9,Orthodox,193.04,203.20,265.0,41.0,38.0


In [8]:
colunas = list(data_heavy_util)

In [9]:
colunas.remove('R_fighter')
colunas.remove('B_fighter')
colunas.remove('title_bout')
colunas.remove('weight_class')
colunas.remove('no_of_rounds')
colunas.remove('B_total_rounds_fought')
colunas.remove('R_total_rounds_fought')

In [10]:
dataf = data_heavy_util.loc[:,colunas]
dataf.head()

,Winner,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,0,0,1,0,17.00,14.500,2.500,2.000,201.000,59.5,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
60,1,0,1,0,8.00,8.000,15.000,13.000,85.000,38.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,1,2,0,0,5.00,3.500,5.500,3.500,16.000,5.5,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,0,0,1,0,35.00,21.000,38.000,30.000,167.000,69.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0
75,1,0,2,0,4.25,3.875,5.375,4.125,26.625,10.5,...,6,0,0,9,Orthodox,193.04,203.20,265.0,41.0,38.0


### Correlações

In [11]:
data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)

In [12]:
blue_winners_correlation = data_heavy_blue_winners.corr()
blue_winners_correlation.head(2)

,date,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
date,1.000000,-0.161123,-0.032762,0.054294,-0.145747,NaN,0.274744,0.256441,0.154872,0.179436,...,0.231107,0.255263,-0.078268,-0.051332,0.205434,0.029779,0.010070,0.050460,0.102656,0.258130
title_bout,-0.161123,1.000000,0.491461,-0.137166,0.345926,NaN,0.000122,0.002365,0.059811,0.056089,...,0.198632,0.182796,0.064792,-0.056728,0.181773,-0.018567,0.100335,-0.185558,0.014377,-0.040564


### Aplicando filtros

In [13]:
filtros = ['R_fighter','B_fighter','Winner','weight_class','title_bout','R_wins','R_losses','B_wins','B_losses',
           'B_Reach_cms','B_Height_cms','B_Weight_lbs','R_Reach_cms','R_Height_cms','R_Weight_lbs']

In [17]:
data_heavy_filtered = data_heavy_util.loc[:,filtros]

In [18]:
data_heavy_filtered.head()

,R_fighter,B_fighter,Winner,weight_class,title_bout,R_wins,R_losses,B_wins,B_losses,B_Reach_cms,B_Height_cms,B_Weight_lbs,R_Reach_cms,R_Height_cms,R_Weight_lbs
4,Tai Tuivasa,Blagoy Ivanov,0,Heavyweight,0,3,1,1,1,185.42,180.34,250.0,190.50,187.96,264.0
60,Arjan Bhullar,Juan Adams,1,Heavyweight,0,2,1,1,0,203.20,195.58,265.0,190.50,185.42,245.0
63,Greg Hardy,Dmitrii Smoliakov,1,Heavyweight,0,0,1,0,2,187.96,187.96,253.0,203.20,195.58,265.0
69,Andrei Arlovski,Augusto Sakai,0,Heavyweight,0,16,12,1,0,195.58,190.50,265.0,195.58,190.50,240.0
75,Alistair Overeem,Aleksei Oleinik,1,Heavyweight,0,9,6,6,2,203.20,187.96,240.0,203.20,193.04,265.0


In [19]:
data_heavy_filtered['B_Reach_cms'].corr(data_heavy_filtered['Winner'])

-0.10192044189964397

### Modelo inicial

Separando em categóricos

In [22]:
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_heavy_cat = data_heavy_util[categoricas].astype('category')
data_heavy_num = data_heavy_util.drop(categoricas, axis=1).astype('float')

In [23]:
X = data_heavy_num.drop('Winner', axis=1)
y = data_heavy_num['Winner']

Separando os dados em testes e treinamento

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [27]:
model = RandomForestClassifier(n_estimators=10000)

model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
#Fatores que tem o maior peso na decisão da vitória
j=1
lista_j=list()
for i, f in sorted(list(zip(model.feature_importances_, X_train.columns)), reverse=True):
    print( j,'°:',f'{f}: {i}')
    a=str(j)+'°'
    lista_j.append(a)
    j+=1

1 °: B_avg_BODY_landed: 0.01961256808989267
2 °: B_avg_BODY_att: 0.019122198966241757
3 °: R_age: 0.016483393937537425
4 °: B_avg_TOTAL_STR_att: 0.014198901327886494
5 °: R_avg_opp_HEAD_landed: 0.013746062677444729
6 °: B_avg_TOTAL_STR_landed: 0.01320384114409928
7 °: B_avg_HEAD_att: 0.012575359998896418
8 °: B_avg_TD_att: 0.01212782708701681
9 °: B_avg_SIG_STR_att: 0.012113425157406087
10 °: B_avg_GROUND_landed: 0.011796556257697141
11 °: R_avg_opp_CLINCH_att: 0.011791720080138535
12 °: R_avg_opp_CLINCH_landed: 0.01156926358219536
13 °: B_avg_GROUND_att: 0.011343080850861479
14 °: B_avg_SIG_STR_landed: 0.011278883041466738
15 °: B_avg_HEAD_landed: 0.011089866939551952
16 °: B_avg_CLINCH_att: 0.010874919615378164
17 °: B_avg_CLINCH_landed: 0.010708649318507573
18 °: R_avg_SIG_STR_att: 0.010663626502318781
19 °: R_avg_opp_BODY_landed: 0.01065457990313124
20 °: B_avg_opp_SIG_STR_pct: 0.010429218171189168
21 °: R_avg_HEAD_att: 0.0100715668903502
22 °: R_avg_BODY_landed: 0.0100664330693747

In [48]:
data={'Fator':X_train.columns ,'Correlação':model.feature_importances_,}
Fator_por_corr=pd.DataFrame(data)
Fator_por_corr=Fator_por_corr.sort_values(by='Correlação', ascending=False)
Fator_por_corr['Grau de Importância']=lista_j
Fator_por_corr = Fator_por_corr.set_index('Grau de Importância')
Fator_por_corr.head()

,Fator,Correlação
Grau de Importância,,
1°,B_avg_BODY_landed,0.019613
2°,B_avg_BODY_att,0.019122
3°,R_age,0.016483
4°,B_avg_TOTAL_STR_att,0.014199
5°,R_avg_opp_HEAD_landed,0.013746


In [42]:
inuteis = []
uteis = []
for index,row in Fator_por_corr.iterrows():
    if row['Correlação'] >= -0.01 and row['Correlação']<=0.01:
        inuteis.append(row['Fator'])
    else:
        uteis.append(row['Fator'])    